# Test OpenAI Agents SDK
- Implement a workflow to write a daily AI newsletter

In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

10:23:19 | NewsletterAgent.newsletter_agent | INFO | Test info message
10:23:19 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
10:23:19 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# set up state
# session_id = 'test_newsletter_20250923174350688839'
# step_name = 'step_05_cluster_by_topic'
# del session_id

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=True, 
                                 process_since=None)
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, db_path="newsletter_agent.db") 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

🚀 Creating NewsletterAgent...
session_id is not defined
test_newsletter_20250924102323067055


In [6]:
state.get_status()

{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_01_fetch_urls',
  'workflow_complete': False,
  'workflow_status': 'not_started',
  'workflow_status_message': '',
  'progress_percentage': 0.0,
  'max_edits': 2,
  'concurrency': 16},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [7]:
state.get_current_step()


'step_01_fetch_urls'

In [8]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


10:23:36 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Starting check_workflow_status
10:23:36 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Completed check_workflow_status


⏱️  Total execution time: 7.58s
📊 Final result:
Current workflow status:
- Progress: 0.0% (0/9 complete)
- 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch URLs

Step-by-step status:
- Step 1: Fetch Urls — not_started
- Step 2: Filter Urls — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Cluster By Topic — not_started
- Step 6: Rate Articles — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

What would you like me to do next? (Options: run all steps, run a specific step, or resume later.)


In [9]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


10:23:50 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Starting check_workflow_status
10:23:50 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Completed check_workflow_status
10:23:53 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Starting Step 1: Gather URLs
10:23:53 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Cleaning download directory: download/sources
10:23:53 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Successfully cleaned download directory
2025-09-24 10:23:53,055 - fetcher_4998801936 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-09-24 10:23:53,063 - fetcher_4998801936 - INFO - [fetcher_init] Loaded 17 sources: 7 RSS, 9 HTML, 1 API
2025-09-24 10:23:53,064 - fetcher_4998801936 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-09-24 10:23:53,064 - fetcher_4998801936 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=

2025-09-24 10:23:55,883 - fetcher_4998801936 - INFO - Starting scrape_source https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day, Reddit
2025-09-24 10:23:55,883 - fetcher_4998801936 - INFO - scrape_url(https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day)
2025-09-24 10:23:55,884 - fetcher_4998801936 - INFO - scraping https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+

2025-09-24 10:24:04,470 - fetcher_4998801936 - DEBUG - Initial sleep: 10
2025-09-24 10:24:13,183 - fetcher_4998801936 - DEBUG - performed human like actions
2025-09-24 10:24:13,194 - fetcher_4998801936 - INFO - Page URL redirected from https://venturebeat.com/category/ai/ to https://venturebeat.com/category/ai
2025-09-24 10:24:13,204 - fetcher_4998801936 - DEBUG - Found last updated time from document.lastModified: 09/24/2025 16:24:13
2025-09-24 10:24:13,204 - fetcher_4998801936 - DEBUG - Attempting to parse last_updated: '09/24/2025 16:24:13' (type: <class 'str'>)
2025-09-24 10:24:13,205 - fetcher_4998801936 - DEBUG - Parsed datetime: 2025-09-24 16:24:13
2025-09-24 10:24:13,205 - fetcher_4998801936 - DEBUG - Added UTC timezone: 2025-09-24 16:24:13+00:00
2025-09-24 10:24:13,205 - fetcher_4998801936 - DEBUG - Converted to UTC: 2025-09-24 16:24:13+00:00
2025-09-24 10:24:13,206 - fetcher_4998801936 - DEBUG - Formatted last_updated: 2025-09-24T16:24:13Z
2025-09-24 10:24:13,206 - fetcher_49

2025-09-24 10:24:21,234 - fetcher_4998801936 - DEBUG - Converted to UTC: 2025-09-24 16:24:21+00:00
2025-09-24 10:24:21,234 - fetcher_4998801936 - DEBUG - Formatted last_updated: 2025-09-24T16:24:21Z
2025-09-24 10:24:21,234 - fetcher_4998801936 - INFO - Saving HTML to download/sources/WSJ.html
2025-09-24 10:24:21,243 - fetcher_4998801936 - INFO - [fetch_html] Parsing HTML file: download/sources/WSJ.html
2025-09-24 10:24:21,273 - fetcher_4998801936 - INFO - [fetch_html] Parsed HTML file: download/sources/WSJ.html
2025-09-24 10:24:21,274 - fetcher_4998801936 - INFO - [fetch_html] HTML fetch successful for WSJ: 27 articles
2025-09-24 10:24:21,716 - fetcher_4998801936 - DEBUG - performed human like actions
2025-09-24 10:24:21,716 - fetcher_4998801936 - INFO - Scrolling Reddit (1/5)
2025-09-24 10:24:24,122 - fetcher_4998801936 - INFO - Scrolling Reddit (2/5)
2025-09-24 10:24:27,116 - fetcher_4998801936 - INFO - Scrolling Reddit (3/5)
2025-09-24 10:24:28,113 - fetcher_4998801936 - DEBUG - per

,source,url
0,Ars Technica,20
1,Bloomberg,31
2,Business Insider,16
3,FT,102
4,Feedly AI,73
5,Hacker News,30
6,HackerNoon,50
7,New York Times,27
8,NewsAPI,99
9,Reddit,60


,source,title,url,published,rss_summary,id
0,Ars Technica,When “no” means “yes”: Why AI chatbots can’t p...,https://arstechnica.com/ai/2025/09/when-no-mea...,"Tue, 23 Sep 2025 22:23:22 +0000",New study examines how a helpful AI response c...,0
1,Ars Technica,OpenAI and Nvidia’s $100B AI plan will require...,https://arstechnica.com/ai/2025/09/openai-and-...,"Mon, 22 Sep 2025 19:17:28 +0000","""This is a giant project,"" Nvidia CEO said of ...",1
2,Ars Technica,DeepMind AI safety report explores the perils ...,https://arstechnica.com/google/2025/09/deepmin...,"Mon, 22 Sep 2025 18:18:00 +0000",DeepMind releases version 3.0 of its AI Fronti...,2
3,Ars Technica,Science journalists find ChatGPT is bad at sum...,https://arstechnica.com/ai/2025/09/science-jou...,"Fri, 19 Sep 2025 17:10:09 +0000","LLM ""tended to sacrifice accuracy for simplici...",3
4,Ars Technica,AI medical tools found to downplay symptoms of...,https://arstechnica.com/health/2025/09/ai-medi...,"Fri, 19 Sep 2025 13:30:27 +0000",Bias-reflecting LLMs lead to inferior medical ...,4
...,...,...,...,...,...,...
688,NewsAPI,AI breakthrough finds life-saving insights in ...,https://www.sciencedaily.com/releases/2025/09/...,2025-09-23T12:33:51Z,NaN,688
689,NewsAPI,BluSky AI Inc. Signs Letter of Intent to Lease...,https://www.globenewswire.com/news-release/202...,2025-09-23T14:12:00Z,NaN,689
690,NewsAPI,Wall Street strategists chase S&P 500 like few...,https://fortune.com/2025/09/23/wall-street-str...,2025-09-23T12:49:28Z,NaN,690
691,NewsAPI,Larry Ellison’s Oracle set to spearhead U.S. o...,https://fortune.com/2025/09/23/oracle-tiktok-d...,2025-09-23T13:28:18Z,NaN,691


10:24:34 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Completed Step 1: Gathered 693 articles
10:24:36 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Starting check_workflow_status
10:24:36 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Completed check_workflow_status


⏱️  Total execution time: 51.86s
📊 Final result:
Step 1 completed.

Summary:
- Fetched 693 articles from 17 sources (RSS).
- Step 1 status: complete.
- Workflow progress: 11.1% (1/9 complete).
- Next step: Step 2 — Filter URLs (not started).

Would you like me to continue and run Step 2 now?


In [10]:
state 


NewsletterAgentState(headline_data=[{'source': 'Ars Technica', 'title': 'When “no” means “yes”: Why AI chatbots can’t process Persian social etiquette', 'url': 'https://arstechnica.com/ai/2025/09/when-no-means-yes-why-ai-chatbots-cant-process-persian-social-etiquette/', 'published': 'Tue, 23 Sep 2025 22:23:22 +0000', 'rss_summary': 'New study examines how a helpful AI response could become a cultural disaster in Iran.', 'id': 0}, {'source': 'Ars Technica', 'title': 'OpenAI and Nvidia’s $100B AI plan will require power equal to 10 nuclear reactors', 'url': 'https://arstechnica.com/ai/2025/09/openai-and-nvidias-100b-ai-plan-will-require-power-equal-to-10-nuclear-reactors/', 'published': 'Mon, 22 Sep 2025 19:17:28 +0000', 'rss_summary': '"This is a giant project," Nvidia CEO said of new 10-gigawatt AI infrastructure deal.', 'id': 1}, {'source': 'Ars Technica', 'title': 'DeepMind AI safety report explores the perils of “misaligned” AI', 'url': 'https://arstechnica.com/google/2025/09/deepmi

In [11]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,When “no” means “yes”: Why AI chatbots can’t p...,https://arstechnica.com/ai/2025/09/when-no-mea...,"Tue, 23 Sep 2025 22:23:22 +0000",New study examines how a helpful AI response c...,0
1,Ars Technica,OpenAI and Nvidia’s $100B AI plan will require...,https://arstechnica.com/ai/2025/09/openai-and-...,"Mon, 22 Sep 2025 19:17:28 +0000","""This is a giant project,"" Nvidia CEO said of ...",1
2,Ars Technica,DeepMind AI safety report explores the perils ...,https://arstechnica.com/google/2025/09/deepmin...,"Mon, 22 Sep 2025 18:18:00 +0000",DeepMind releases version 3.0 of its AI Fronti...,2
3,Ars Technica,Science journalists find ChatGPT is bad at sum...,https://arstechnica.com/ai/2025/09/science-jou...,"Fri, 19 Sep 2025 17:10:09 +0000","LLM ""tended to sacrifice accuracy for simplici...",3
4,Ars Technica,AI medical tools found to downplay symptoms of...,https://arstechnica.com/health/2025/09/ai-medi...,"Fri, 19 Sep 2025 13:30:27 +0000",Bias-reflecting LLMs lead to inferior medical ...,4
...,...,...,...,...,...,...
688,NewsAPI,AI breakthrough finds life-saving insights in ...,https://www.sciencedaily.com/releases/2025/09/...,2025-09-23T12:33:51Z,NaN,688
689,NewsAPI,BluSky AI Inc. Signs Letter of Intent to Lease...,https://www.globenewswire.com/news-release/202...,2025-09-23T14:12:00Z,NaN,689
690,NewsAPI,Wall Street strategists chase S&P 500 like few...,https://fortune.com/2025/09/23/wall-street-str...,2025-09-23T12:49:28Z,NaN,690
691,NewsAPI,Larry Ellison’s Oracle set to spearhead U.S. o...,https://fortune.com/2025/09/23/oracle-tiktok-d...,2025-09-23T13:28:18Z,NaN,691


In [12]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
3,FT,102
8,NewsAPI,99
4,Feedly AI,73
9,Reddit,60
12,The Register,50
6,HackerNoon,50
1,Bloomberg,31
16,Washington Post,30
5,Hacker News,30
7,New York Times,27


In [13]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


10:25:29 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Starting Step 2: Filter URLs


,source,title,url,published,rss_summary,id
0,Ars Technica,When “no” means “yes”: Why AI chatbots can’t p...,https://arstechnica.com/ai/2025/09/when-no-mea...,"Tue, 23 Sep 2025 22:23:22 +0000",New study examines how a helpful AI response c...,0
1,Ars Technica,OpenAI and Nvidia’s $100B AI plan will require...,https://arstechnica.com/ai/2025/09/openai-and-...,"Mon, 22 Sep 2025 19:17:28 +0000","""This is a giant project,"" Nvidia CEO said of ...",1
2,Ars Technica,DeepMind AI safety report explores the perils ...,https://arstechnica.com/google/2025/09/deepmin...,"Mon, 22 Sep 2025 18:18:00 +0000",DeepMind releases version 3.0 of its AI Fronti...,2
3,Ars Technica,Science journalists find ChatGPT is bad at sum...,https://arstechnica.com/ai/2025/09/science-jou...,"Fri, 19 Sep 2025 17:10:09 +0000","LLM ""tended to sacrifice accuracy for simplici...",3
4,Ars Technica,AI medical tools found to downplay symptoms of...,https://arstechnica.com/health/2025/09/ai-medi...,"Fri, 19 Sep 2025 13:30:27 +0000",Bias-reflecting LLMs lead to inferior medical ...,4
...,...,...,...,...,...,...
688,NewsAPI,AI breakthrough finds life-saving insights in ...,https://www.sciencedaily.com/releases/2025/09/...,2025-09-23T12:33:51Z,NaN,688
689,NewsAPI,BluSky AI Inc. Signs Letter of Intent to Lease...,https://www.globenewswire.com/news-release/202...,2025-09-23T14:12:00Z,NaN,689
690,NewsAPI,Wall Street strategists chase S&P 500 like few...,https://fortune.com/2025/09/23/wall-street-str...,2025-09-23T12:49:28Z,NaN,690
691,NewsAPI,Larry Ellison’s Oracle set to spearhead U.S. o...,https://fortune.com/2025/09/23/oracle-tiktok-d...,2025-09-23T13:28:18Z,NaN,691


INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=458, user_len=954
10:25:46 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Completed Step 2: 355 AI-related articles


⏱️  Total execution time: 16.71s
📊 Final result:
✅ Step 2 completed successfully! Filtered 693 headlines to 355 AI-related articles.

📊 Results stored in persistent state. Current step: step_03_download_articles


In [14]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

10:25:57 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Starting Step 3: Download Articles
10:25:57 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Starting concurrent scraping of 355 AI-related articles
10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Launching browser for 355 URLs with 16 concurrent workers
10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 0 fetching 1 of 355 https://www.ft.com/content/d2913fba-a867-4b63-9fed-1dd2e1c65453
10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | scrape_url(https://www.ft.com/content/d2913fba-a867-4b63-9fed-1dd2e1c65453)
10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | scraping https://www.ft.com/content/d2913fba-a867-4b63-9fed-1dd2e1c65453 to download/html
10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Downloading https://www.ft.com/content/d2913fba-a867-4b63-9fed-1dd2e1c65453
10:25:5

10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | File already exists: download/html/Terminators__AI-driven_robot_war_machines_on_the_march.html
10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 8 completed https://go.theregister.com/feed/www.theregister.com/2025/09/12/terminators_aidriven_robot_war_machines/ with status: success
10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 8 fetching 13 of 355 https://www.bloomberg.com/news/articles/2025-09-23/microsoft-msft-is-turning-to-the-field-of-microfluidics-to-cool-down-ai-chips
10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Skipping ignored domain: www.bloomberg.com
10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 8 completed https://www.bloomberg.com/news/articles/2025-09-23/microsoft-msft-is-turning-to-the-field-of-microfluidics-to-cool-down-ai-chips with status: success
10:25:58 | NewsletterAgent.

10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | scraping https://www.globenewswire.com/news-release/2025/09/23/3154863/0/en/RealReports-and-Restb-ai-introduce-industry-first-AI-property-intelligence-to-MLSs.html to download/html
10:25:58 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Downloading https://www.globenewswire.com/news-release/2025/09/23/3154863/0/en/RealReports-and-Restb-ai-introduce-industry-first-AI-property-intelligence-to-MLSs.html
10:25:59 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Response: 200
10:25:59 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Response: 200
10:25:59 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Response: 200
10:26:00 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Response: 200
10:26:00 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 11 fetching 24 of 355 https://www.washingtonpost.com/technology/2025/09/16/sen

10:26:30 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Downloading https://financialpost.com/pmn/business-wire-news-releases-pmn/new-wolters-kluwer-survey-finds-86-of-north-american-finance-teams-in-early-stages-of-ai-adoption
10:26:30 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 12 completed https://hackernoon.com/revering-ai-reveals-incompetence-not-intelligence?source=rss with status: success
10:26:30 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 12 fetching 31 of 355 https://hackernoon.com/moonbirds-and-azuki-ip-coming-to-verse8-as-ai-native-game-platform-integrates-with-story?source=rss
10:26:30 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | File already exists: download/html/Moonbirds_and_Azuki_IP_Coming_To_Verse8_as_AI-Native_Game_Platform_Integrates_With_Story.html
10:26:30 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 12 completed https://hackernoon.com/moonbirds-and-

10:26:37 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 1 fetching 40 of 355 https://www.investors.com/market-trend/stock-market-today/dow-jones-sp500-nasdaq-new-home-sales-micron-stock-mu/
10:26:37 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | scrape_url(https://www.investors.com/market-trend/stock-market-today/dow-jones-sp500-nasdaq-new-home-sales-micron-stock-mu/)
10:26:37 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | scraping https://www.investors.com/market-trend/stock-market-today/dow-jones-sp500-nasdaq-new-home-sales-micron-stock-mu/ to download/html
10:26:37 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Downloading https://www.investors.com/market-trend/stock-market-today/dow-jones-sp500-nasdaq-new-home-sales-micron-stock-mu/
10:26:39 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Response: 200
10:26:44 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Response: 200
10:2

...

10:46:13 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 12 fetching 347 of 355 https://www.bloomberg.com/news/features/2025-07-30/startup-builder-ai-goes-from-1-5-billion-unicorn-to-bankruptcy
10:46:13 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Skipping ignored domain: www.bloomberg.com
10:46:13 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 12 completed https://www.bloomberg.com/news/features/2025-07-30/startup-builder-ai-goes-from-1-5-billion-unicorn-to-bankruptcy with status: success
10:46:13 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 12 fetching 348 of 355 https://www.washingtonpost.com/technology/2025/09/15/openai-chatgpt-study-use-cases/
10:46:13 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | File already exists: download/html/Here_s_what_the_data_says_people_ask_ChatGPTOpenAI_released_the_first_detailed_public_study_on_who_uses_its_chatbot_and_what_they_most_often_as

10:47:25 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 10 completed https://www.techradar.com/pro/what-is-alert-fatigue-and-how-can-ai-powered-naas-reduce-it with status: success
10:47:27 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Saving HTML to download/html/To_build_a_better_Internet_in_the_age_of_AI__we_need_responsible_AI_bot_principles._Here_s_our_proposal..html
10:47:27 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Worker 3 completed https://blog.cloudflare.com/building-a-better-internet-with-responsible-ai-bot-principles/ with status: success
10:47:28 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Page URL redirected from https://go.theregister.com/feed/www.theregister.com/2025/09/23/selfdriving_car_fooled_with_mirrors/ to https://www.theregister.com/2025/09/23/selfdriving_car_fooled_with_mirrors/
10:47:28 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Saving HTML to download/html/B

Starting with 322 rows...
Processing 322 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2

10:48:37 | NewsletterAgent.test_newsletter_20250924102323067055 | INFO | Completed Step 3: Downloaded 323 articles


Filtering dataframe...
  Pair: 314 vs 326
    314: NewsAPI - Factbox-Companies pouring billions to advance AI infrastructure
    326: NewsAPI - Factbox-Companies pouring billions to advance AI infrastructure
  Pair: 345 vs 304
    345: NewsAPI - The AI boom is unsustainable unless tech spending goes ‘parabolic,’ Deutsche Bank warns: ‘This is highly unlikely’
    304: NewsAPI - The AI boom is unsustainable unless tech spending goes ‘parabolic,’ Deutsche Bank warns: ‘This is highly unlikely’
  Pair: 342 vs 341
    342: NewsAPI - Paytm Money launches India’s first AI-driven equity fund with JioBlackRock
    341: NewsAPI - Paytm Money launches India’s first AI-driven equity fund with JioBlackRock
  Pair: 315 vs 324
    315: NewsAPI - Brazil's WEG to invest $77 million to boost US plant capacity due to AI demand
    324: NewsAPI - Brazil's WEG to invest $77 million to boost US plant capacity due to AI demand
  Pair: 135 vs 205
    135: Feedly AI - Sources: Oracle seeks to borrow $15B throug

In [15]:
import db


In [16]:
#     initial_url: str
#     final_url: str
#     title: str
#     isAI: bool
#     created_at: Optional[datetime]
headline_df = agent.state.headline_dict.copy()

In [17]:
headline_df

,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length
0,Feedly AI,UAE firms plan to invest $100 million in AI st...,https://www.semafor.com/article/09/24/2025/uae...,NaN,NaN,229,True,success,https://www.semafor.com/article/09/24/2025/uae...,download/html/UAE_firms_plan_to_invest__100_mi...,2025-09-24T12:09:05Z,download/text/UAE_firms_plan_to_invest__100_mi...,30247
1,FT,Companies’ legal teams feel the AI spark,https://www.ft.com/content/e4609b0a-15f1-4256-...,NaN,NaN,156,True,success,https://www.ft.com/content/e4609b0a-15f1-4256-...,download/html/Companies__legal_teams_feel_the_...,2025-09-19T04:00:11Z,download/text/Companies__legal_teams_feel_the_...,26984
2,Hacker News,Getting AI to work in complex codebases,https://github.com/humanlayer/advanced-context...,"Tue, 23 Sep 2025 14:27:36 +0000","<a href=""https://news.ycombinator.com/item?id=...",267,True,success,https://github.com/humanlayer/advanced-context...,download/html/Getting_AI_to_work_in_complex_co...,None,download/text/Getting_AI_to_work_in_complex_co...,21451
3,HackerNoon,New frontiers in Human AI Interface,https://hackernoon.com/new-frontiers-in-human-...,"Fri, 19 Sep 2025 08:14:10 GMT",Recent tech advances are breaking free from 20...,303,True,success,https://hackernoon.com/new-frontiers-in-human-...,download/html/New_frontiers_in_Human_AI_Interf...,None,download/text/New_frontiers_in_Human_AI_Interf...,20864
4,Feedly AI,"To build a better Internet in the age of AI, w...",https://blog.cloudflare.com/building-a-better-...,NaN,NaN,190,True,success,https://blog.cloudflare.com/building-a-better-...,download/html/To_build_a_better_Internet_in_th...,2025-09-24T13:00:00Z,download/text/To_build_a_better_Internet_in_th...,19097
...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,Bloomberg,Jack Ma Returns With a Vengeance to ‘Make Alib...,https://www.bloomberg.com/news/features/2025-0...,NaN,NaN,30,True,success,https://www.bloomberg.com/news/features/2025-0...,,None,,0
311,Bloomberg,Markets WrapStock Futures Rise as Alibaba Boos...,https://www.bloomberg.com/news/articles/2025-0...,NaN,NaN,35,True,success,https://www.bloomberg.com/news/articles/2025-0...,,None,,0
312,Bloomberg,AI Wants More Data. More Chips. More Real Esta...,https://www.bloomberg.com/news/features/2024-1...,NaN,NaN,36,True,success,https://www.bloomberg.com/news/features/2024-1...,,None,,0
313,Bloomberg,US Stocks Resume Rally as Alibaba AI Spending ...,https://www.bloomberg.com/news/articles/2025-0...,NaN,NaN,20,True,success,https://www.bloomberg.com/news/articles/2025-0...,,None,,0


In [25]:
import sqlite3
import db

with sqlite3.connect("newsletter_agent.db") as conn:
    db.Url.create_table(conn)
    for row in headline_df.itertuples():
        print(row.url, '', row.title, row.isAI, datetime.now())
        myurl = db.Url(row.url, '', row.title, row.isAI, datetime.now())
        myurl.insert(conn)


https://www.semafor.com/article/09/24/2025/uae-firms-plan-100-million-investments-in-ai-startups-worldwide  UAE firms plan to invest $100 million in AI startups worldwide True 2025-09-24 11:21:10.985556
https://www.ft.com/content/e4609b0a-15f1-4256-8056-3364d991ec59  Companies’ legal teams feel the AI spark True 2025-09-24 11:21:10.987789
https://github.com/humanlayer/advanced-context-engineering-for-coding-agents/blob/main/ace-fca.md  Getting AI to work in complex codebases True 2025-09-24 11:21:10.988722
https://hackernoon.com/new-frontiers-in-human-ai-interface?source=rss  New frontiers in Human AI Interface True 2025-09-24 11:21:10.989123
https://blog.cloudflare.com/building-a-better-internet-with-responsible-ai-bot-principles/  To build a better Internet in the age of AI, we need responsible AI bot principles. Here's our proposal. True 2025-09-24 11:21:10.989730
https://spectrum.ieee.org/ai-impact-on-job-market  Is AI Making Entry-Level Tech Jobs Disappear? True 2025-09-24 11:21:1

In [ ]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
# User prompt to run workflow
# user_prompt = "Run step 5, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


In [ ]:
state.headline_dict.loc[state.headline_dict["url"] != state.headline_dict["final_url"]]


In [ ]:
state.headline_dict.columns

In [ ]:

    def create_extended_summary(row):
        parts = []

        # Add title if present
        if 'title' in row and row['title']:
            parts.append(str(row['title']).strip())

        # Add description if present
        if 'description' in row and row['description']:
            parts.append(str(row['description']).strip())

        # Add topics if present (join with commas)
        if 'topics' in row and row['topics']:
            if isinstance(row['topics'], list):
                topics_str = ", ".join(str(topic).strip() for topic in row['topics'] if topic)
            else:
                topics_str = str(row['topics']).strip()
            if topics_str:
                parts.append(topics_str)

        # Add summary if present
        if pd.notna(row.get('summary')) and row.get('summary'):
            parts.append(str(row['summary']).strip())

        return "\n\n".join(parts)

    async def _get_embeddings_df(self, headline_data: pd.DataFrame, embedding_model: str = "text-embedding-3-large") -> pd.DataFrame:
        """
        Get embeddings for article summaries and return as DataFrame.

        Args:
            headline_data: DataFrame with articles containing summary column
            embedding_model: OpenAI embedding model to use

        Returns:
            DataFrame with embeddings for each extended summary
        """
        from openai import OpenAI
        from llm import paginate_df_async

        # Create extended_summary column by concatenating available fields
        headline_data_copy = headline_data.copy()

        headline_data_copy['extended_summary'] = headline_data_copy.apply(create_extended_summary, axis=1)

        # Filter to articles with non-empty extended summaries
        articles_with_summaries = headline_data_copy[
            (headline_data_copy['extended_summary'].notna()) &
            (headline_data_copy['extended_summary'] != '')
        ].copy()

        all_embeddings = []
        client = OpenAI()

        # Use paginate_df_async similar to dedupe_by_cosine_similarity.py
        async for batch_df in paginate_df_async(articles_with_summaries, 25):
            text_batch = batch_df["extended_summary"].to_list()
            response = client.embeddings.create(input=text_batch, model=embedding_model)
            batch_embeddings = [item.embedding for item in response.data]
            all_embeddings.extend(batch_embeddings)

        # Create DataFrame with embeddings, preserving original index
        embedding_df = pd.DataFrame(
            all_embeddings,
            index=articles_with_summaries.index
        )

        return embedding_df


In [ ]:
headline_df = state.headline_dict
headline_df['extended_summary'] = headline_df.apply(create_extended_summary, axis=1)


embeddings_df = await _get_embeddings_df(_, state.headline_dict)

In [ ]:
embeddings_df

In [ ]:
n_components = 60
min_cluster_size = 4
min_samples =3 



In [ ]:
from sklearn.decomposition import TruncatedSVD
RANDOM_STATE = 42

svd = TruncatedSVD(n_components=n_components, random_state=RANDOM_STATE)
reduced_embeddings = svd.fit_transform(embeddings_df)
# Re-normalize after SVD
reduced_embeddings /= np.linalg.norm(reduced_embeddings, axis=1, keepdims=True)


In [ ]:
# Fit HDBSCAN
print("=== HDBSCAN Parameters ===")
print(f"min_cluster_size:   {min_cluster_size}")
print(f"min_samples:        {min_samples}")
print(f"n_components:       {n_components}")
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=min_cluster_size,
    min_samples=min_samples,
    metric="euclidean",
    cluster_selection_method="eom",
)

labels = clusterer.fit_predict(reduced_embeddings)



In [ ]:
def calculate_clustering_metrics(embeddings_array, labels, clusterer=None):
    """
    Calculate various clustering quality metrics for HDBSCAN results.
    
    Args:
        embeddings_array: Original normalized embeddings used for clustering
        labels: Cluster labels from HDBSCAN
        clusterer: Optional HDBSCAN clusterer object
    
    Returns:
        Dictionary of clustering metrics
    """
    
    # Filter out noise points (-1 labels) for some metrics
    non_noise_mask = labels != -1
    non_noise_embeddings = embeddings_array[non_noise_mask]
    non_noise_labels = labels[non_noise_mask]
    
    metrics = {}
    
    # Basic cluster statistics
    unique_labels = set(labels)
    n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
    n_noise = np.sum(labels == -1)
    
    metrics['n_clusters'] = n_clusters
    metrics['n_noise_points'] = n_noise
    metrics['noise_ratio'] = n_noise / len(labels)
    
    # Cluster size distribution
    cluster_sizes = Counter(labels[labels != -1])
    if cluster_sizes:
        metrics['avg_cluster_size'] = np.mean(list(cluster_sizes.values()))
        metrics['std_cluster_size'] = np.std(list(cluster_sizes.values()))
        metrics['min_cluster_size'] = min(cluster_sizes.values())
        metrics['max_cluster_size'] = max(cluster_sizes.values())
    
    # Skip other metrics if we have too few clusters or too much noise
    if n_clusters < 2 or len(non_noise_labels) < 2:
        print("Warning: Too few clusters or too much noise for some metrics")
        return metrics
    
    # HDBSCAN-specific metrics
    # gives some divide by 0 errors
    if clusterer is not None:
        try:
            # Validity index (HDBSCAN's internal metric)
            validity_idx = hdbscan.validity.validity_index(
                embeddings_array, labels, metric='euclidean'
            )
            metrics['hdbscan_validity_index'] = validity_idx
        except Exception as e:
            print(f"Could not compute HDBSCAN validity index: {e}")
        
        # Cluster persistence (stability)
        if hasattr(clusterer, 'cluster_persistence_'):
            metrics['cluster_persistence'] = clusterer.cluster_persistence_
    
    # Scikit-learn clustering metrics (excluding noise points)
    try:
        # Silhouette Score (higher is better, range [-1, 1])
        sil_score = silhouette_score(non_noise_embeddings, non_noise_labels, metric='euclidean')
        metrics['silhouette_score'] = sil_score
        
        # Calinski-Harabasz Index (higher is better)
        ch_score = calinski_harabasz_score(non_noise_embeddings, non_noise_labels)
        metrics['calinski_harabasz_score'] = ch_score
        
        # Davies-Bouldin Index (lower is better)
        db_score = davies_bouldin_score(non_noise_embeddings, non_noise_labels)
        metrics['davies_bouldin_score'] = db_score
        
    except Exception as e:
        print(f"Could not compute sklearn metrics: {e}")
    
    # Custom composite score balancing cluster quality and quantity
    if 'silhouette_score' in metrics and n_clusters > 0:
        # Penalize too many small clusters or too few large clusters
        cluster_balance = 1 / (1 + abs(np.log(n_clusters / 10)))  # Optimal around 10 clusters
        size_consistency = 1 / (1 + metrics.get('std_cluster_size', 0) / max(metrics.get('avg_cluster_size', 1), 1))
        noise_penalty = 1 - min(metrics['noise_ratio'], 0.5)  # Penalize high noise
        
        composite_score = (
            0.5 * max(metrics['silhouette_score'], 0) +  # Quality component
            0.5 * max(metrics['hdbscan_validity_index'], 0)
#             0.1 * cluster_balance +                       # Quantity component  
#             0.1 * size_consistency +                      # Size consistency
#             0.3 * noise_penalty                           # Noise penalty
        )
        metrics['composite_score'] = composite_score
    
    return metrics

def print_clustering_summary(metrics):
    """Print a nice summary of clustering metrics."""
    print("=== Clustering Quality Metrics ===")
    print(f"Number of clusters: {metrics.get('n_clusters', 'N/A')}")
    print(f"Noise points: {metrics.get('n_noise_points', 'N/A')} ({metrics.get('noise_ratio', 0):.1%})")
    
    if 'avg_cluster_size' in metrics:
        print(f"Average cluster size: {metrics['avg_cluster_size']:.1f} ± {metrics.get('std_cluster_size', 0):.1f}")
        print(f"Cluster size range: {metrics.get('min_cluster_size', 'N/A')} - {metrics.get('max_cluster_size', 'N/A')}")
    
    print("=== Quality Scores ===")
    if 'silhouette_score' in metrics:
        print(f"Silhouette Score: {metrics['silhouette_score']:.3f} (higher is better)")
    if 'calinski_harabasz_score' in metrics:
        print(f"Calinski-Harabasz Score: {metrics['calinski_harabasz_score']:.1f} (higher is better)")
    if 'davies_bouldin_score' in metrics:
        print(f"Davies-Bouldin Score: {metrics['davies_bouldin_score']:.3f} (lower is better)")
    if 'hdbscan_validity_index' in metrics:
        print(f"HDBSCAN Validity Index: {metrics['hdbscan_validity_index']:.3f}")
    if 'composite_score' in metrics:
        print(f"Composite Score: {metrics['composite_score']:.3f} (higher is better)")
    print()



In [ ]:
from collections import Counter
import optuna

# Calculate metrics
metrics = calculate_clustering_metrics(reduced_embeddings, labels, clusterer)
print_clustering_summary(metrics)


In [ ]:
MIN_COMPONENTS = 20
def objective(trial, embeddings_array):

    n_components = trial.suggest_int('n_components', 
                                     MIN_COMPONENTS, 
                                     embeddings_array.shape[1] // 4)  
    
    svd = TruncatedSVD(n_components=n_components, random_state=RANDOM_STATE)
    reduced_embeddings = svd.fit_transform(embeddings_array)
    # Re-normalize after SVD
    reduced_embeddings /= np.linalg.norm(reduced_embeddings, axis=1, keepdims=True)

    # HDBSCAN hyperparameters to optimize
    min_cluster_size = trial.suggest_int('min_cluster_size', 2, 10)
    min_samples = trial.suggest_int('min_samples', 2, min_cluster_size)

    # Fit HDBSCAN
    print("=== HDBSCAN Parameters ===")
    print(f"min_cluster_size:   {min_cluster_size}")
    print(f"min_samples:        {min_samples}")
    print(f"n_components:       {n_components}")
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric="euclidean",
        cluster_selection_method="eom",
    )

    labels = clusterer.fit_predict(reduced_embeddings)

    # Calculate metrics
    metrics = calculate_clustering_metrics(reduced_embeddings, labels, clusterer)
    print_clustering_summary(metrics)

    # Return negative composite score (Optuna minimizes)
    composite_score = metrics.get('composite_score', -1.0)

    # Penalize if no valid clusters found or too much noise
    if metrics.get('n_clusters', 0) < 2 or metrics.get('noise_ratio', 1.0) > 0.8:
        composite_score = -1.0

    return -composite_score    



In [ ]:
def optimize_hdbscan(embeddings_array, n_trials=100, timeout=None):
    """
    Optimize HDBSCAN hyperparameters using Optuna.
    
    Args:
        embeddings_array: Normalized embeddings array
        n_trials: Number of optimization trials
        timeout: Maximum time in seconds (None for no limit)
    
    Returns:
        Dictionary with best parameters and results
    """
    
    print(f"Starting optimization with {n_trials} trials...")
    print(f"Original embedding shape: {embeddings_array.shape}")
    
    # Create study
    study = optuna.create_study(
        direction='minimize',  # We return negative composite score
        sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE),
        pruner=optuna.pruners.MedianPruner(n_startup_trials=10)
    )
    
    # Optimize
    study.optimize(
        lambda trial: objective(trial, embeddings_array),
        n_trials=n_trials,
        timeout=timeout,
        show_progress_bar=True
    )
    
    # Get best parameters
    best_params = study.best_params
    best_score = -study.best_value  # Convert back to positive
    
    print(f"\nOptimization completed!")
    print(f"Best composite score: {best_score:.4f}")
    print(f"Best parameters: {best_params}")
    
    # Test best parameters
    print(f"\n=== Results with Best Parameters ===")
        
    # Apply best dimensionality reduction
    if best_params['n_components'] < embeddings_array.shape[1]:
        svd = TruncatedSVD(n_components=n_components, random_state=RANDOM_STATE)
        best_embeddings = svd.fit_transform(embeddings_array)
        # Re-normalize after SVD
        best_embeddings /= np.linalg.norm(reduced_embeddings, axis=1, keepdims=True)
        print(f"Reduced dimensions from {embeddings_array.shape[1]} to {best_params['n_components']}")
    else:
        best_embeddings = embeddings_array
        reducer = None
        print("No dimensionality reduction applied")
     
    # Fit with best parameters
    best_clusterer = hdbscan.HDBSCAN(
        min_cluster_size=best_params['min_cluster_size'],
        min_samples=best_params['min_samples'],
        metric="euclidean",
        cluster_selection_method="eom",
    )
    
    best_labels = best_clusterer.fit_predict(best_embeddings)
    best_metrics = calculate_clustering_metrics(best_embeddings, best_labels, best_clusterer)
    
    print_clustering_summary(best_metrics)
    print()
    
    # Return results
    return {
        'study': study,
        'best_params': best_params,
        'best_score': best_score,
        'best_clusterer': best_clusterer,
        'best_labels': best_labels,
        'best_embeddings': best_embeddings,
        'best_metrics': best_metrics,
        'svd_transformer': svd if best_params['n_components'] < embeddings_array.shape[1] else None
    }

results = optimize_hdbscan(embeddings_df, n_trials=100)

In [ ]:
results


In [ ]:

metrics = calculate_clustering_metrics(embeddings_df.values, labels, clusterer) 
print_clustering_summary(metrics)


In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_proampt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

- cluster  articles
- combine title, description, topics, summary if present 
- fetch embeddings for summaries 
- do dimensionality reduction
- cluster with hdbscan
- show metrics

- tune dbscan
- name the clusters with topic_writer
- store the cluster names 

output is , df has topic list and topic_str, summary updated, df has cluster , state clusters updated

In [ ]:
state = await agent.run_step("get state")
state 


In [ ]:
inspect_result = await agent.run_step("inspect state")


In [ ]:
state = await agent.get_state_direct()


In [ ]:
print(status_result)
